In [1]:
# import libraries
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [2]:
# use option to display all columns of dataframe
pd.set_option('display.max_columns', None)

In [3]:
# import the data using URL
# currently there is an error with the data
#github_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/merged_gw.csv"
#raw_data = pd.read_csv(github_url)

In [4]:
# import the data using local file
raw_data = pd.read_csv("merged_gw.csv")

### Data Exploration

In [5]:
raw_data.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,modified,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,77,0.01,0.01,0.0,1.02,6,1,0,3.6,22.8,2024-08-17T14:00:00Z,62,False,16,0,0,0,0,1,0,4339,1,1,1,0,2,0,0,0,50,False,0,1
1,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,2.2,0,0,0,0,0.0,427,0.00,0.00,0.0,0.00,6,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,3,0,0,0,0,1,0,33324,0,1,1,0,0,0,0,0,45,True,0,1
2,Tomiyasu Takehiro,DEF,Arsenal,0.0,0,0,0,0,0.0,22,0.00,0.00,0.0,0.00,2,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,20,0,0,0,0,1,0,8462,0,0,2,0,0,0,0,0,50,True,0,1
3,Malcolm Ebiowei,MID,Crystal Palace,0.0,0,0,0,0,0.0,197,0.00,0.00,0.0,0.00,8,0,0,0.0,0.0,2024-08-18T13:00:00Z,0,False,4,0,0,0,0,1,0,716,0,1,2,0,0,0,0,0,45,False,0,1
4,Ben Brereton Díaz,MID,Southampton,1.0,0,0,-2,0,14.0,584,0.02,0.32,0.3,0.25,5,1,0,3.3,2.6,2024-08-17T14:00:00Z,70,False,15,0,0,0,0,1,0,66244,1,0,1,16,1,0,0,0,55,False,1,1


In [6]:
# explore the first row of data to get an idea of how it looks
raw_data.iloc[0]

name                                    Alex Scott
position                                       MID
team                                   Bournemouth
xP                                             1.6
assists                                          0
bonus                                            0
bps                                             11
clean_sheets                                     0
creativity                                    12.8
element                                         77
expected_assists                              0.01
expected_goal_involvements                    0.01
expected_goals                                 0.0
expected_goals_conceded                       1.02
fixture                                          6
goals_conceded                                   1
goals_scored                                     0
ict_index                                      3.6
influence                                     22.8
kickoff_time                  2

In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15663 entries, 0 to 15662
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        15663 non-null  object 
 1   position                    15663 non-null  object 
 2   team                        15663 non-null  object 
 3   xP                          15663 non-null  float64
 4   assists                     15663 non-null  int64  
 5   bonus                       15663 non-null  int64  
 6   bps                         15663 non-null  int64  
 7   clean_sheets                15663 non-null  int64  
 8   creativity                  15663 non-null  float64
 9   element                     15663 non-null  int64  
 10  expected_assists            15663 non-null  float64
 11  expected_goal_involvements  15663 non-null  float64
 12  expected_goals              15663 non-null  float64
 13  expected_goals_conceded     156

In [8]:
raw_data.describe()

,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,1.566300e+04,15663.000000,15663.000000,15663.000000,15663.000000,15663.000000,1.566300e+04,1.566300e+04,1.566300e+04,15663.000000,15663.000000,15663.000000
mean,1.115348,0.038626,0.092511,4.976186,0.075273,4.613746,343.057843,0.028375,0.072023,0.043653,0.474827,118.359637,0.485284,0.042968,1.552461,6.720411,28.798251,10.506225,0.001149,0.000511,0.000511,0.001979,12.286216,0.092894,2.324573e+05,0.321650,1.522250,1.519760,4.203792,1.202260,3.728877e+01,1.772259e+04,1.768493e+04,49.432931,0.064355,12.286216
std,1.769651,0.210749,0.455037,9.593121,0.263840,10.526526,198.809970,0.086722,0.191907,0.150669,0.778667,66.459569,0.945196,0.225453,2.823981,12.624999,38.509345,5.765637,0.035716,0.022595,0.022595,0.044445,6.638185,0.635328,6.366596e+05,0.467124,1.272681,1.219642,10.930053,2.377174,7.925094e+04,6.724457e+04,6.108365e+04,10.593248,0.245393,6.638185
min,-2.200000,0.000000,0.000000,-25.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-5.000000,-2.741008e+06,0.000000e+00,0.000000e+00,5.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,171.000000,0.000000,0.000000,0.000000,0.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,5.435000e+03,0.000000,1.000000,1.000000,0.000000,0.000000,-1.839000e+03,5.400000e+01,2.200000e+02,44.000000,0.000000,7.000000
50%,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,343.000000,0.000000,0.000000,0.000000,0.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,2.372800e+04,0.000000,1.000000,1.000000,0.000000,0.000000,-1.550000e+02,5.390000e+02,1.463000e+03,46.000000,0.000000,12.000000
75%,1.800000,0.000000,0.000000,6.000000,0.000000,2.400000,513.000000,0.010000,0.030000,0.000000,0.800000,176.500000,1.000000,0.000000,2.200000,9.000000,76.000000,15.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,1.547855e+05,1.000000,2.000000,2.000000,2.000000,1.000000,2.200000e+01,5.550500e+03,9.663000e+03,52.000000,0.000000,18.000000
max,15.500000,3.000000,3.000000,97.000000,1.000000,114.300000,756.000000,1.230000,3.030000,2.410000,5.430000,230.000000,6.000000,4.000000,28.600000,149.000000,90.000000,20.000000,2.000000,1.000000,1.000000,1.000000,23.000000,13.000000,7.952602e+06,1.000000,6.000000,5.000000,120.000000,25.000000,1.216262e+06,1.305135e+06,2.766073e+06,154.000000,1.000000,23.000000


In [9]:
raw_data.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW'],
      dtype='object')

### Data Cleaning/Preprocessing

In [10]:
columns_filter = ['GW', 'round', 'fixture', 'name', 'position', 'team', 'opponent_team', 'starts', 'minutes',
                    'goals_scored', 'expected_goals', 'assists', 'expected_assists', 'expected_goal_involvements',
                    'goals_conceded', 'expected_goals_conceded', 'clean_sheets', 'own_goals',
                    'yellow_cards', 'red_cards', 'penalties_missed', 'saves', 'penalties_saved', 'was_home',
                    'total_points','bonus']
df = raw_data[columns_filter].copy()
df.head()

,GW,round,fixture,name,position,team,opponent_team,starts,minutes,goals_scored,expected_goals,assists,expected_assists,expected_goal_involvements,goals_conceded,expected_goals_conceded,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,was_home,total_points,bonus
0,1,1,6,Alex Scott,MID,Bournemouth,16,1,62,0,0.0,0,0.01,0.01,1,1.02,0,0,0,0,0,0,0,False,2,0
1,1,1,6,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,3,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0
2,1,1,2,Tomiyasu Takehiro,DEF,Arsenal,20,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0
3,1,1,8,Malcolm Ebiowei,MID,Crystal Palace,4,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,False,0,0
4,1,1,5,Ben Brereton Díaz,MID,Southampton,15,1,70,0,0.3,0,0.02,0.32,1,0.25,0,0,1,0,0,0,0,False,1,0


In [11]:
# Create a team ID, team Name dictionary using a dictionary comprehension and enumerate function
# Enumerate function prints the values and their index, starting from 0 unless specified (here it is specified to start at 1)
teams_id_dict = {id : team for id, team in enumerate(df['team'].sort_values().unique(), 1)}
teams_id_dict

{1: 'Arsenal',
 2: 'Aston Villa',
 3: 'Bournemouth',
 4: 'Brentford',
 5: 'Brighton',
 6: 'Chelsea',
 7: 'Crystal Palace',
 8: 'Everton',
 9: 'Fulham',
 10: 'Ipswich',
 11: 'Leicester',
 12: 'Liverpool',
 13: 'Man City',
 14: 'Man Utd',
 15: 'Newcastle',
 16: "Nott'm Forest",
 17: 'Southampton',
 18: 'Spurs',
 19: 'West Ham',
 20: 'Wolves'}

In [12]:
# Create a new column which gives the name of the opponent team, rather than using the ID
df['opponent'] = df['opponent_team'].map(teams_id_dict)

In [13]:
df = df.rename(columns={'goals_scored' : 'goals',
                        'expected_goals' : 'xG',
                        'expected_assists' : 'xA',
                        'expected_goal_involvements' : 'xGI',
                        'goals_conceded' : 'GC',
                        'expected_goals_conceded' : 'xGC',
                        'total_points' : 'points'})

In [14]:
fpl = df.copy()

In [15]:
# create new columns for goals and expected goals home and away
fpl["goals_home"] = np.where(fpl["was_home"] == True, df["goals"], 0)
fpl["goals_away"] = np.where(fpl["was_home"] == False, df["goals"], 0)
fpl["xG_home"] = np.where(fpl["was_home"] == True, df["xG"], 0)
fpl["xG_away"] = np.where(fpl["was_home"] == False, df["xG"], 0)

In [16]:
fpl.head()

,GW,round,fixture,name,position,team,opponent_team,starts,minutes,goals,xG,assists,xA,xGI,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,was_home,points,bonus,opponent,goals_home,goals_away,xG_home,xG_away
0,1,1,6,Alex Scott,MID,Bournemouth,16,1,62,0,0.0,0,0.01,0.01,1,1.02,0,0,0,0,0,0,0,False,2,0,Nott'm Forest,0,0,0.0,0.0
1,1,1,6,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,3,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Bournemouth,0,0,0.0,0.0
2,1,1,2,Tomiyasu Takehiro,DEF,Arsenal,20,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Wolves,0,0,0.0,0.0
3,1,1,8,Malcolm Ebiowei,MID,Crystal Palace,4,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,False,0,0,Brentford,0,0,0.0,0.0
4,1,1,5,Ben Brereton Díaz,MID,Southampton,15,1,70,0,0.3,0,0.02,0.32,1,0.25,0,0,1,0,0,0,0,False,1,0,Newcastle,0,0,0.0,0.3


### Teams Stats Tables

Gameweek Team Stats

In [17]:
gw_team_grouped = fpl.groupby(['GW', 'round', 'fixture', 'team', 'opponent']).aggregate({'goals': 'sum',
                                                                                        'xG': 'sum',
                                                                                        'was_home': 'max',
                                                                                        'goals_home' : 'sum',
                                                                                        'goals_away' : 'sum',
                                                                                        'xG_home' : 'sum',
                                                                                        'xG_away' : 'sum'}).reset_index()
gw_team_grouped

,GW,round,fixture,team,opponent,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away
0,1,1,1,Fulham,Man Utd,0,0.44,False,0,0,0.00,0.44
1,1,1,1,Man Utd,Fulham,1,2.43,True,1,0,2.43,0.00
2,1,1,2,Arsenal,Wolves,2,1.26,True,2,0,1.26,0.00
3,1,1,2,Wolves,Arsenal,0,0.48,False,0,0,0.00,0.48
4,1,1,3,Brighton,Everton,3,1.44,False,0,3,0.00,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,Newcastle,1,0.76,True,1,0,0.76,0.00
454,23,23,229,Leicester,Spurs,2,1.54,False,0,2,0.00,1.54
455,23,23,229,Spurs,Leicester,1,1.04,True,1,0,1.04,0.00
456,23,23,230,Arsenal,Wolves,1,1.00,False,0,1,0.00,1.00


Join the grouped DataFrame with itself to get the stats of the opponent team such as goals conceded, expected goals conceded - do not sum these columns on a player basis as each player will have the total goals conceded and xGC they faced specifically and does not reflect the entire team, plus players who got subbed off early may have lower stats that what the entire team should get

In [18]:
# join the grouped df with itself to get the stats of the opponent team, which will give the goals conceded, xGC, etc. of the team in question
gw_team_stats_merged = pd.merge(gw_team_grouped,
                         gw_team_grouped,
                         how='left',
                         left_on=['GW', 'round', 'fixture', 'team'],
                         right_on=['GW', 'round', 'fixture', 'opponent'],
                         suffixes=('', '_opponent'))
gw_team_stats_merged

,GW,round,fixture,team,opponent,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away,team_opponent,opponent_opponent,goals_opponent,xG_opponent,was_home_opponent,goals_home_opponent,goals_away_opponent,xG_home_opponent,xG_away_opponent
0,1,1,1,Fulham,Man Utd,0,0.44,False,0,0,0.00,0.44,Man Utd,Fulham,1,2.43,True,1,0,2.43,0.00
1,1,1,1,Man Utd,Fulham,1,2.43,True,1,0,2.43,0.00,Fulham,Man Utd,0,0.44,False,0,0,0.00,0.44
2,1,1,2,Arsenal,Wolves,2,1.26,True,2,0,1.26,0.00,Wolves,Arsenal,0,0.48,False,0,0,0.00,0.48
3,1,1,2,Wolves,Arsenal,0,0.48,False,0,0,0.00,0.48,Arsenal,Wolves,2,1.26,True,2,0,1.26,0.00
4,1,1,3,Brighton,Everton,3,1.44,False,0,3,0.00,1.44,Everton,Brighton,0,0.46,True,0,0,0.46,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,Newcastle,1,0.76,True,1,0,0.76,0.00,Newcastle,Southampton,3,2.91,False,0,3,0.00,2.91
454,23,23,229,Leicester,Spurs,2,1.54,False,0,2,0.00,1.54,Spurs,Leicester,1,1.04,True,1,0,1.04,0.00
455,23,23,229,Spurs,Leicester,1,1.04,True,1,0,1.04,0.00,Leicester,Spurs,2,1.54,False,0,2,0.00,1.54
456,23,23,230,Arsenal,Wolves,1,1.00,False,0,1,0.00,1.00,Wolves,Arsenal,0,0.75,True,0,0,0.75,0.00


In [19]:
gw_team_stats_sliced = gw_team_stats_merged[['GW', 'round', 'fixture', 'team',
                                      'goals','xG', 'was_home', 'goals_home', 'goals_away', 'xG_home', 'xG_away',
                                      'goals_opponent', 'xG_opponent', 'goals_away_opponent', 'goals_home_opponent',
                                      'xG_away_opponent', 'xG_home_opponent', 'opponent']].copy()

team_stats_per_gw = gw_team_stats_sliced.rename(columns={'goals_opponent': 'GC',
                                                         'xG_opponent': 'xGC',
                                                         'goals_home_opponent' : 'GC_away',
                                                         'goals_away_opponent' : 'GC_home',
                                                         'xG_home_opponent' : 'xGC_away',
                                                         'xG_away_opponent' : 'xGC_home'})
team_stats_per_gw

,GW,round,fixture,team,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away,GC,xGC,GC_home,GC_away,xGC_home,xGC_away,opponent
0,1,1,1,Fulham,0,0.44,False,0,0,0.00,0.44,1,2.43,0,1,0.00,2.43,Man Utd
1,1,1,1,Man Utd,1,2.43,True,1,0,2.43,0.00,0,0.44,0,0,0.44,0.00,Fulham
2,1,1,2,Arsenal,2,1.26,True,2,0,1.26,0.00,0,0.48,0,0,0.48,0.00,Wolves
3,1,1,2,Wolves,0,0.48,False,0,0,0.00,0.48,2,1.26,0,2,0.00,1.26,Arsenal
4,1,1,3,Brighton,3,1.44,False,0,3,0.00,1.44,0,0.46,0,0,0.00,0.46,Everton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,1,0.76,True,1,0,0.76,0.00,3,2.91,3,0,2.91,0.00,Newcastle
454,23,23,229,Leicester,2,1.54,False,0,2,0.00,1.54,1,1.04,0,1,0.00,1.04,Spurs
455,23,23,229,Spurs,1,1.04,True,1,0,1.04,0.00,2,1.54,2,0,1.54,0.00,Leicester
456,23,23,230,Arsenal,1,1.00,False,0,1,0.00,1.00,0,0.75,0,0,0.00,0.75,Wolves


In [20]:
# create a new column for clean sheets
team_stats_per_gw['CS'] = np.where(team_stats_per_gw['GC'] == 0, 1, 0)
team_stats_per_gw['CS_home'] = np.where((team_stats_per_gw['GC'] == 0) & (team_stats_per_gw['was_home'] == True), 1, 0)
team_stats_per_gw['CS_away'] = np.where((team_stats_per_gw['GC'] == 0) & (team_stats_per_gw['was_home'] == False), 1, 0)

# create a new column for whether they won the match or not
team_stats_per_gw['match_won'] = np.where(team_stats_per_gw['goals'] > team_stats_per_gw['GC'], 1, 0)
#team_stats_per_gw['match_won_home'] = np.where((team_stats_per_gw['goals'] > team_stats_per_gw['GC']) & (team_stats_per_gw['was_home'] == True), 1, 0)
#team_stats_per_gw['match_won_away'] = np.where((team_stats_per_gw['goals'] > team_stats_per_gw['GC']) & (team_stats_per_gw['was_home'] == False), 1, 0)

# potentially use this
team_stats_per_gw['match_won_home'] = np.where((team_stats_per_gw['goals_home'] > team_stats_per_gw['GC_home']), 1, 0)
team_stats_per_gw['match_won_away'] = np.where((team_stats_per_gw['goals_away'] > team_stats_per_gw['GC_away']), 1, 0)
team_stats_per_gw

,GW,round,fixture,team,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away,GC,xGC,GC_home,GC_away,xGC_home,xGC_away,opponent,CS,CS_home,CS_away,match_won,match_won_home,match_won_away
0,1,1,1,Fulham,0,0.44,False,0,0,0.00,0.44,1,2.43,0,1,0.00,2.43,Man Utd,0,0,0,0,0,0
1,1,1,1,Man Utd,1,2.43,True,1,0,2.43,0.00,0,0.44,0,0,0.44,0.00,Fulham,1,1,0,1,1,0
2,1,1,2,Arsenal,2,1.26,True,2,0,1.26,0.00,0,0.48,0,0,0.48,0.00,Wolves,1,1,0,1,1,0
3,1,1,2,Wolves,0,0.48,False,0,0,0.00,0.48,2,1.26,0,2,0.00,1.26,Arsenal,0,0,0,0,0,0
4,1,1,3,Brighton,3,1.44,False,0,3,0.00,1.44,0,0.46,0,0,0.00,0.46,Everton,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,1,0.76,True,1,0,0.76,0.00,3,2.91,3,0,2.91,0.00,Newcastle,0,0,0,0,0,0
454,23,23,229,Leicester,2,1.54,False,0,2,0.00,1.54,1,1.04,0,1,0.00,1.04,Spurs,0,0,0,1,0,1
455,23,23,229,Spurs,1,1.04,True,1,0,1.04,0.00,2,1.54,2,0,1.54,0.00,Leicester,0,0,0,0,0,0
456,23,23,230,Arsenal,1,1.00,False,0,1,0.00,1.00,0,0.75,0,0,0.00,0.75,Wolves,1,0,1,1,0,1


In [21]:
team_total_stats = team_stats_per_gw.groupby('team').agg({'goals' : 'sum',
                                                         'xG' : 'sum',
                                                         'goals_home' : 'sum',
                                                         'xG_home' : 'sum',
                                                         'goals_away' : 'sum',
                                                         'xG_away' : 'sum',
                                                         'GC' : 'sum',
                                                         'xGC' : 'sum',
                                                         'GC_home' : 'sum',
                                                         'xGC_home' : 'sum',
                                                         'GC_away' : 'sum',
                                                         'xGC_away' : 'sum',
                                                         'CS' : 'sum',
                                                         'CS_home' : 'sum',
                                                         'CS_away' : 'sum',
                                                         'fixture' : 'count',
                                                         'was_home' : 'sum',
                                                         'match_won' : 'sum',
                                                         'match_won_home' : 'sum',
                                                         'match_won_away' : 'sum'}).reset_index()

team_total_stats = team_total_stats.rename(columns={'fixture' : 'games_played',
                                                    'was_home' : 'games_played_home',
                                                    'match_won' : 'games_won',
                                                    'match_won_home' : 'games_won_home',
                                                    'match_won_away' : 'games_won_away'})

team_total_stats['games_played_away'] = team_total_stats['games_played'] - team_total_stats['games_played_home']
team_total_stats

,team,goals,xG,goals_home,xG_home,goals_away,xG_away,GC,xGC,GC_home,xGC_home,GC_away,xGC_away,CS,CS_home,CS_away,games_played,games_played_home,games_won,games_won_home,games_won_away,games_played_away
0,Arsenal,42,38.99,20,21.43,22,17.56,21,20.57,9,6.89,12,13.68,8,5,3,23,11,12,6,6,12
1,Aston Villa,33,34.59,20,21.07,13,13.52,35,28.74,14,9.66,21,19.08,3,2,1,23,12,10,6,4,11
2,Bournemouth,41,44.34,17,21.21,24,23.13,26,29.09,7,11.11,19,17.98,6,5,1,23,11,11,6,5,12
3,Brentford,41,37.02,28,22.71,13,14.31,39,39.62,22,19.79,17,19.83,4,1,3,23,12,8,6,2,11
4,Brighton,34,31.95,13,16.55,21,15.40,30,33.14,14,14.24,16,18.90,5,2,3,23,11,8,3,5,12
5,Chelsea,45,47.21,20,23.36,25,23.85,30,32.91,14,12.72,16,20.19,4,1,3,23,11,11,5,6,12
6,Crystal Palace,25,31.87,11,16.86,14,15.01,28,32.76,18,16.57,10,16.19,8,3,5,23,12,7,3,4,11
7,Everton,16,22.38,9,12.03,7,10.35,28,29.58,13,14.57,15,15.01,8,4,4,22,11,4,2,2,11
8,Fulham,33,32.71,17,15.68,16,17.03,30,26.99,17,13.98,13,13.01,4,1,3,23,12,8,4,4,11
9,Ipswich,20,21.04,8,10.99,12,10.05,46,48.72,24,21.96,22,26.76,2,1,1,23,12,2,1,1,11


In [22]:
#team_total_stats.to_csv("team_total_stats.csv", index=False)

Need to make all stats on a **per game** basis as Everton and Liverpool have played 1 game less.

Also need to make stats on a **per home/away game** basis so that it can be ranked fairly instead of doing total_home/total_away

In [23]:
stats_columns = list(team_total_stats.columns)
items_to_remove = ['team', 'games_played', 'games_played_home', 'games_played_away']

for item in items_to_remove:
    stats_columns.remove(item)
print(stats_columns)

['goals', 'xG', 'goals_home', 'xG_home', 'goals_away', 'xG_away', 'GC', 'xGC', 'GC_home', 'xGC_home', 'GC_away', 'xGC_away', 'CS', 'CS_home', 'CS_away', 'games_won', 'games_won_home', 'games_won_away']


In [24]:
for column in stats_columns:
    if '_home' in column:
        team_total_stats[f'{column}_pg_home'] = round(team_total_stats[column] / team_total_stats['games_played_home'], 2)
    elif '_away' in column:
        team_total_stats[f'{column}_pg_away'] = round(team_total_stats[column] / team_total_stats['games_played_away'], 2)
    else:
        team_total_stats[f'{column}_pg'] = round(team_total_stats[column] / team_total_stats['games_played'], 2)
team_total_stats

,team,goals,xG,goals_home,xG_home,goals_away,xG_away,GC,xGC,GC_home,xGC_home,GC_away,xGC_away,CS,CS_home,CS_away,games_played,games_played_home,games_won,games_won_home,games_won_away,games_played_away,goals_pg,xG_pg,goals_home_pg_home,xG_home_pg_home,goals_away_pg_away,xG_away_pg_away,GC_pg,xGC_pg,GC_home_pg_home,xGC_home_pg_home,GC_away_pg_away,xGC_away_pg_away,CS_pg,CS_home_pg_home,CS_away_pg_away,games_won_pg,games_won_home_pg_home,games_won_away_pg_away
0,Arsenal,42,38.99,20,21.43,22,17.56,21,20.57,9,6.89,12,13.68,8,5,3,23,11,12,6,6,12,1.83,1.70,1.82,1.95,1.83,1.46,0.91,0.89,0.82,0.63,1.00,1.14,0.35,0.45,0.25,0.52,0.55,0.50
1,Aston Villa,33,34.59,20,21.07,13,13.52,35,28.74,14,9.66,21,19.08,3,2,1,23,12,10,6,4,11,1.43,1.50,1.67,1.76,1.18,1.23,1.52,1.25,1.17,0.80,1.91,1.73,0.13,0.17,0.09,0.43,0.50,0.36
2,Bournemouth,41,44.34,17,21.21,24,23.13,26,29.09,7,11.11,19,17.98,6,5,1,23,11,11,6,5,12,1.78,1.93,1.55,1.93,2.00,1.93,1.13,1.26,0.64,1.01,1.58,1.50,0.26,0.45,0.08,0.48,0.55,0.42
3,Brentford,41,37.02,28,22.71,13,14.31,39,39.62,22,19.79,17,19.83,4,1,3,23,12,8,6,2,11,1.78,1.61,2.33,1.89,1.18,1.30,1.70,1.72,1.83,1.65,1.55,1.80,0.17,0.08,0.27,0.35,0.50,0.18
4,Brighton,34,31.95,13,16.55,21,15.40,30,33.14,14,14.24,16,18.90,5,2,3,23,11,8,3,5,12,1.48,1.39,1.18,1.50,1.75,1.28,1.30,1.44,1.27,1.29,1.33,1.58,0.22,0.18,0.25,0.35,0.27,0.42
5,Chelsea,45,47.21,20,23.36,25,23.85,30,32.91,14,12.72,16,20.19,4,1,3,23,11,11,5,6,12,1.96,2.05,1.82,2.12,2.08,1.99,1.30,1.43,1.27,1.16,1.33,1.68,0.17,0.09,0.25,0.48,0.45,0.50
6,Crystal Palace,25,31.87,11,16.86,14,15.01,28,32.76,18,16.57,10,16.19,8,3,5,23,12,7,3,4,11,1.09,1.39,0.92,1.40,1.27,1.36,1.22,1.42,1.50,1.38,0.91,1.47,0.35,0.25,0.45,0.30,0.25,0.36
7,Everton,16,22.38,9,12.03,7,10.35,28,29.58,13,14.57,15,15.01,8,4,4,22,11,4,2,2,11,0.73,1.02,0.82,1.09,0.64,0.94,1.27,1.34,1.18,1.32,1.36,1.36,0.36,0.36,0.36,0.18,0.18,0.18
8,Fulham,33,32.71,17,15.68,16,17.03,30,26.99,17,13.98,13,13.01,4,1,3,23,12,8,4,4,11,1.43,1.42,1.42,1.31,1.45,1.55,1.30,1.17,1.42,1.16,1.18,1.18,0.17,0.08,0.27,0.35,0.33,0.36
9,Ipswich,20,21.04,8,10.99,12,10.05,46,48.72,24,21.96,22,26.76,2,1,1,23,12,2,1,1,11,0.87,0.91,0.67,0.92,1.09,0.91,2.00,2.12,2.00,1.83,2.00,2.43,0.09,0.08,0.09,0.09,0.08,0.09


In [25]:
# extract only the per game columns
per_game_totals_columns = [column for column in team_total_stats.columns if '_pg' in column]
per_game_totals_df = team_total_stats[per_game_totals_columns].copy()
per_game_totals_df = per_game_totals_df.set_index(team_total_stats['team'])
per_game_totals_df

,goals_pg,xG_pg,goals_home_pg_home,xG_home_pg_home,goals_away_pg_away,xG_away_pg_away,GC_pg,xGC_pg,GC_home_pg_home,xGC_home_pg_home,GC_away_pg_away,xGC_away_pg_away,CS_pg,CS_home_pg_home,CS_away_pg_away,games_won_pg,games_won_home_pg_home,games_won_away_pg_away
team,,,,,,,,,,,,,,,,,,
Arsenal,1.83,1.70,1.82,1.95,1.83,1.46,0.91,0.89,0.82,0.63,1.00,1.14,0.35,0.45,0.25,0.52,0.55,0.50
Aston Villa,1.43,1.50,1.67,1.76,1.18,1.23,1.52,1.25,1.17,0.80,1.91,1.73,0.13,0.17,0.09,0.43,0.50,0.36
Bournemouth,1.78,1.93,1.55,1.93,2.00,1.93,1.13,1.26,0.64,1.01,1.58,1.50,0.26,0.45,0.08,0.48,0.55,0.42
Brentford,1.78,1.61,2.33,1.89,1.18,1.30,1.70,1.72,1.83,1.65,1.55,1.80,0.17,0.08,0.27,0.35,0.50,0.18
Brighton,1.48,1.39,1.18,1.50,1.75,1.28,1.30,1.44,1.27,1.29,1.33,1.58,0.22,0.18,0.25,0.35,0.27,0.42
Chelsea,1.96,2.05,1.82,2.12,2.08,1.99,1.30,1.43,1.27,1.16,1.33,1.68,0.17,0.09,0.25,0.48,0.45,0.50
Crystal Palace,1.09,1.39,0.92,1.40,1.27,1.36,1.22,1.42,1.50,1.38,0.91,1.47,0.35,0.25,0.45,0.30,0.25,0.36
Everton,0.73,1.02,0.82,1.09,0.64,0.94,1.27,1.34,1.18,1.32,1.36,1.36,0.36,0.36,0.36,0.18,0.18,0.18
Fulham,1.43,1.42,1.42,1.31,1.45,1.55,1.30,1.17,1.42,1.16,1.18,1.18,0.17,0.08,0.27,0.35,0.33,0.36


### Player Stats Tables

Should make a stats per GW dataframe as will need this for future predictions, as well as a total table for each player

In [26]:
fpl_players = fpl.copy()
fpl_players.head()

,GW,round,fixture,name,position,team,opponent_team,starts,minutes,goals,xG,assists,xA,xGI,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,was_home,points,bonus,opponent,goals_home,goals_away,xG_home,xG_away
0,1,1,6,Alex Scott,MID,Bournemouth,16,1,62,0,0.0,0,0.01,0.01,1,1.02,0,0,0,0,0,0,0,False,2,0,Nott'm Forest,0,0,0.0,0.0
1,1,1,6,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,3,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Bournemouth,0,0,0.0,0.0
2,1,1,2,Tomiyasu Takehiro,DEF,Arsenal,20,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Wolves,0,0,0.0,0.0
3,1,1,8,Malcolm Ebiowei,MID,Crystal Palace,4,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,False,0,0,Brentford,0,0,0.0,0.0
4,1,1,5,Ben Brereton Díaz,MID,Southampton,15,1,70,0,0.3,0,0.02,0.32,1,0.25,0,0,1,0,0,0,0,False,1,0,Newcastle,0,0,0.0,0.3


In [27]:
fpl_players['appearance'] = np.where(fpl_players['minutes'] > 0, 1, 0)

In [28]:
fpl_players['GI'] = fpl_players['goals'] + fpl_players['assists']

In [29]:
# should maybe make fields for home and away for all player stats too
# maybe in the final table I can have a column for the coalesce of home or away stats depending on the fixture, as well as the total stat

In [30]:
# Include stats for the previous 6 games to show form maybe? Might be useful for predicting future performance
# This will be done by grouping the data by player and then using the rolling function to get the sum of the previous 6 games

In [31]:
player_fields = ['appearance','starts','minutes','goals','xG', 'assists','xA', 'GI','xGI', 'GC','xGC', 'clean_sheets', 'own_goals',
                 'yellow_cards', 'red_cards', 'penalties_missed','saves', 'penalties_saved', 'bonus','points']
# CAUTION - xGI and GI will be highly correlated with goals and assists, so maybe remove this

In [32]:
player_stats = fpl_players.groupby(['name', 'team', 'position']).sum()[player_fields].reset_index()
player_stats.head(10)

,name,team,position,appearance,starts,minutes,goals,xG,assists,xA,GI,xGI,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points
0,Aaron Cresswell,West Ham,DEF,9,2,204,0,0.03,0,0.38,0,0.41,2,3.42,0,0,2,0,0,0,0,0,8
1,Aaron Hickey,Brentford,DEF,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0,0,0,0,0
2,Aaron Ramsdale,Arsenal,GK,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0,0,0,0,0
3,Aaron Ramsdale,Southampton,GK,15,15,1350,0,0.00,0,0.02,0,0.02,33,34.92,2,0,1,0,0,62,0,5,46
4,Aaron Wan-Bissaka,West Ham,DEF,22,21,1894,2,0.97,2,1.57,4,2.54,42,37.45,3,0,1,0,0,0,0,6,64
5,Abdoulaye Doucouré,Everton,MID,20,19,1591,0,0.91,0,0.61,0,1.52,21,24.05,8,0,2,0,0,0,0,0,44
6,Abdukodir Khusanov,Man City,DEF,1,1,53,0,0.02,0,0.00,0,0.02,1,1.69,0,0,1,0,0,0,0,0,0
7,Abdul Fatawu,Leicester,MID,11,6,576,0,0.41,2,1.42,2,1.83,14,11.00,0,0,0,0,0,0,0,3,26
8,Adam Armstrong,Southampton,FWD,20,15,1246,2,3.28,3,0.56,5,3.84,35,33.47,2,0,4,0,1,0,0,2,45
9,Adam Lallana,Southampton,MID,12,5,324,0,0.15,2,0.48,2,0.63,7,7.03,0,0,4,0,0,0,0,1,15


In [33]:
per_game_cols = ["starts", "minutes", "goals", "xG", "assists", "xA", "GI", "xGI", "GC", "xGC", "clean_sheets", "own_goals", "yellow_cards",
                 "red_cards", "penalties_missed", "saves", "penalties_saved", "points"]

for column in per_game_cols:
    #player_stats[column + "_p90"] = round((player_stats[column]/player_stats["minutes"])*90, 2) # don't think I want to use per 90 stats
    player_stats[column + "_pg"] = round((player_stats[column]/player_stats["appearance"]), 2)

player_stats.head()

,name,team,position,appearance,starts,minutes,goals,xG,assists,xA,GI,xGI,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points,starts_pg,minutes_pg,goals_pg,xG_pg,assists_pg,xA_pg,GI_pg,xGI_pg,GC_pg,xGC_pg,clean_sheets_pg,own_goals_pg,yellow_cards_pg,red_cards_pg,penalties_missed_pg,saves_pg,penalties_saved_pg,points_pg
0,Aaron Cresswell,West Ham,DEF,9,2,204,0,0.03,0,0.38,0,0.41,2,3.42,0,0,2,0,0,0,0,0,8,0.22,22.67,0.00,0.00,0.00,0.04,0.00,0.05,0.22,0.38,0.00,0.0,0.22,0.0,0.0,0.00,0.0,0.89
1,Aaron Hickey,Brentford,DEF,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron Ramsdale,Arsenal,GK,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aaron Ramsdale,Southampton,GK,15,15,1350,0,0.00,0,0.02,0,0.02,33,34.92,2,0,1,0,0,62,0,5,46,1.00,90.00,0.00,0.00,0.00,0.00,0.00,0.00,2.20,2.33,0.13,0.0,0.07,0.0,0.0,4.13,0.0,3.07
4,Aaron Wan-Bissaka,West Ham,DEF,22,21,1894,2,0.97,2,1.57,4,2.54,42,37.45,3,0,1,0,0,0,0,6,64,0.95,86.09,0.09,0.04,0.09,0.07,0.18,0.12,1.91,1.70,0.14,0.0,0.05,0.0,0.0,0.00,0.0,2.91


In [34]:
# remove players who did not make any appearance
player_stats = player_stats.loc[(player_stats['appearance'] > 0)].copy()

In [35]:
player_stats = player_stats.rename(columns={'appearance' : 'total_appearances',
                                            'starts' : 'total_starts',
                                            'minutes' : 'total_minutes',
                                            'goals' : 'total_goals',
                                            'xG' : 'total_xG',
                                            'assists' : 'total_assists',
                                            'xA' : 'total_xA',
                                            'GI' : 'total_GI',
                                            'xGI' : 'total_xGI',
                                            'GC' : 'total_GC',
                                            'xGC' : 'total_xGC',
                                            'clean_sheets' : 'total_CS',
                                            'own_goals' : 'total_own_goals',
                                            'yellow_cards' : 'total_yellow_cards',
                                            'red_cards' : 'total_red_cards',
                                            'penalties_missed' : 'total_penalties_missed',
                                            'saves' : 'total_saves',
                                            'penalties_saved' : 'total_penalties_saved',
                                            'bonus' : 'total_bonus',
                                            'points' : 'total_points'})
player_stats.head()

,name,team,position,total_appearances,total_starts,total_minutes,total_goals,total_xG,total_assists,total_xA,total_GI,total_xGI,total_GC,total_xGC,total_CS,total_own_goals,total_yellow_cards,total_red_cards,total_penalties_missed,total_saves,total_penalties_saved,total_bonus,total_points,starts_pg,minutes_pg,goals_pg,xG_pg,assists_pg,xA_pg,GI_pg,xGI_pg,GC_pg,xGC_pg,clean_sheets_pg,own_goals_pg,yellow_cards_pg,red_cards_pg,penalties_missed_pg,saves_pg,penalties_saved_pg,points_pg
0,Aaron Cresswell,West Ham,DEF,9,2,204,0,0.03,0,0.38,0,0.41,2,3.42,0,0,2,0,0,0,0,0,8,0.22,22.67,0.00,0.00,0.00,0.04,0.00,0.05,0.22,0.38,0.00,0.0,0.22,0.0,0.0,0.00,0.0,0.89
3,Aaron Ramsdale,Southampton,GK,15,15,1350,0,0.00,0,0.02,0,0.02,33,34.92,2,0,1,0,0,62,0,5,46,1.00,90.00,0.00,0.00,0.00,0.00,0.00,0.00,2.20,2.33,0.13,0.0,0.07,0.0,0.0,4.13,0.0,3.07
4,Aaron Wan-Bissaka,West Ham,DEF,22,21,1894,2,0.97,2,1.57,4,2.54,42,37.45,3,0,1,0,0,0,0,6,64,0.95,86.09,0.09,0.04,0.09,0.07,0.18,0.12,1.91,1.70,0.14,0.0,0.05,0.0,0.0,0.00,0.0,2.91
5,Abdoulaye Doucouré,Everton,MID,20,19,1591,0,0.91,0,0.61,0,1.52,21,24.05,8,0,2,0,0,0,0,0,44,0.95,79.55,0.00,0.05,0.00,0.03,0.00,0.08,1.05,1.20,0.40,0.0,0.10,0.0,0.0,0.00,0.0,2.20
6,Abdukodir Khusanov,Man City,DEF,1,1,53,0,0.02,0,0.00,0,0.02,1,1.69,0,0,1,0,0,0,0,0,0,1.00,53.00,0.00,0.02,0.00,0.00,0.00,0.02,1.00,1.69,0.00,0.0,1.00,0.0,0.0,0.00,0.0,0.00


### Data Preparation for Model

Need to join the total player, team and opponent stats on a gameweek level

This table will be used to make predictions for future points because when making predictions on future games, we will not have stats such as goals scored, assists, total points, xG, xA, xGI, etc. for the specific match as they did not happen. Therefore, only aggregated columns should be used and total points should be predicted based on the total player, team, and opponent stats.

**OR**

Can the model be trained on all stats and predictions are used from a selection of the variables?? Like for a Regression Model, you can just use the coefficients that you will have available

In [36]:
# For the model, do I need to set the index as the player name, team, opponent, etc.? That way I can map the predicted points to a player for a fixture?

In [37]:
df.head()

,GW,round,fixture,name,position,team,opponent_team,starts,minutes,goals,xG,assists,xA,xGI,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,was_home,points,bonus,opponent
0,1,1,6,Alex Scott,MID,Bournemouth,16,1,62,0,0.0,0,0.01,0.01,1,1.02,0,0,0,0,0,0,0,False,2,0,Nott'm Forest
1,1,1,6,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,3,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Bournemouth
2,1,1,2,Tomiyasu Takehiro,DEF,Arsenal,20,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Wolves
3,1,1,8,Malcolm Ebiowei,MID,Crystal Palace,4,0,0,0,0.0,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,False,0,0,Brentford
4,1,1,5,Ben Brereton Díaz,MID,Southampton,15,1,70,0,0.3,0,0.02,0.32,1,0.25,0,0,1,0,0,0,0,False,1,0,Newcastle


In [38]:
# remove rows where the player played no minutes
df = df.loc[(df['minutes'] > 0)].reset_index(drop=True).copy()

In [39]:
model_df = df.copy()
model_df = model_df[['GW', 'fixture', 'name', 'position', 'team', 'opponent', 'starts', 'minutes', 'goals', 'xG', 'assists', 'xA',
                     'GC', 'xGC', 'clean_sheets', 'own_goals', 'yellow_cards', 'red_cards', 'penalties_missed', 'saves', 'penalties_saved', 'bonus', 'points']].copy()
model_df

,GW,fixture,name,position,team,opponent,starts,minutes,goals,xG,assists,xA,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points
0,1,6,Alex Scott,MID,Bournemouth,Nott'm Forest,1,62,0,0.00,0,0.01,1,1.02,0,0,0,0,0,0,0,0,2
1,1,5,Ben Brereton Díaz,MID,Southampton,Newcastle,1,70,0,0.30,0,0.02,1,0.25,0,0,1,0,0,0,0,0,1
2,1,7,Pau Torres,DEF,Aston Villa,West Ham,1,90,0,0.00,0,0.01,1,2.46,0,0,0,0,0,0,0,0,2
3,1,2,Hwang Hee-chan,MID,Wolves,Arsenal,1,90,0,0.00,0,0.11,2,1.24,0,0,0,0,0,0,0,0,2
4,1,2,João Victor Gomes da Silva,MID,Wolves,Arsenal,1,83,0,0.00,0,0.02,2,1.24,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6941,23,224,Romain Esse,MID,Crystal Palace,Brentford,0,7,1,0.32,0,0.00,0,0.29,0,0,0,0,0,0,0,2,8
6942,23,225,Raúl Jiménez,FWD,Fulham,Man Utd,1,63,0,0.08,0,0.00,0,0.24,1,0,0,0,0,0,0,0,2
6943,23,230,Myles Lewis-Skelly,MID,Arsenal,Wolves,1,42,0,0.00,0,0.03,0,0.42,0,0,0,1,0,0,0,0,-2
6944,23,223,Ashley Young,DEF,Everton,Brighton,0,12,0,0.00,0,0.00,0,0.06,0,0,0,0,0,0,0,0,1


In [40]:
# use per game stats for players to reduce bias towards players who have played more games
# keep the total_apperances column for the model so that the model can still take into account total games played for a player
pg_player_stats = player_stats[['name', 'team', 'position', 'total_appearances', 'minutes_pg', 'goals_pg', 'xG_pg', 'assists_pg', 'xA_pg',
                                'GC_pg', 'xGC_pg', 'clean_sheets_pg', 'own_goals_pg', 'yellow_cards_pg', 'red_cards_pg','penalties_missed_pg',
                                'saves_pg', 'penalties_saved_pg', 'points_pg']].copy()

In [41]:
# add a minimum games played filter to remove players who have not played a certain amount of games
minimum_games_threshold = 5
pg_player_stats = pg_player_stats.loc[(pg_player_stats['total_appearances'] >= minimum_games_threshold)].reset_index(drop=True).copy()

In [42]:
# using an inner join to only include players who have played a minimum amount of games
# this is because players who have played less games will have a higher bias towards their per game stats
gw_player_joined = pd.merge(model_df, pg_player_stats, how='inner', on=['name', 'team', 'position'], suffixes=(None, '_player'))
gw_player_joined

,GW,fixture,name,position,team,opponent,starts,minutes,goals,xG,assists,xA,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points,total_appearances,minutes_pg,goals_pg,xG_pg,assists_pg,xA_pg,GC_pg,xGC_pg,clean_sheets_pg,own_goals_pg,yellow_cards_pg,red_cards_pg,penalties_missed_pg,saves_pg,penalties_saved_pg,points_pg
0,1,6,Alex Scott,MID,Bournemouth,Nott'm Forest,1,62,0,0.00,0,0.01,1,1.02,0,0,0,0,0,0,0,0,2,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38
1,2,12,Alex Scott,MID,Bournemouth,Newcastle,0,26,0,0.02,0,0.01,1,1.36,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38
2,3,24,Alex Scott,MID,Bournemouth,Everton,0,24,0,0.00,0,0.01,0,0.26,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38
3,4,32,Alex Scott,MID,Bournemouth,Chelsea,0,22,0,0.00,0,0.00,1,0.32,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38
4,5,46,Alex Scott,MID,Bournemouth,Liverpool,0,21,0,0.10,0,0.02,0,0.40,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,19,187,Jakub Stolarczyk,GK,Leicester,Man City,1,90,0,0.00,0,0.00,2,1.27,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67
6735,20,191,Jakub Stolarczyk,GK,Leicester,Aston Villa,1,90,0,0.00,0,0.00,2,1.56,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67
6736,21,205,Jakub Stolarczyk,GK,Leicester,Crystal Palace,1,90,0,0.00,0,0.00,2,1.82,0,0,0,0,0,2,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67
6737,22,216,Jakub Stolarczyk,GK,Leicester,Fulham,1,90,0,0.00,0,0.00,2,2.19,0,0,0,0,0,0,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67


In [43]:
# need to use per game stats for teams because not all teams have played equal number of games
pg_team_stats = team_total_stats[['team', 'goals_pg', 'xG_pg', 'GC_pg', 'xGC_pg', 'CS_pg', 'games_won_pg']].copy()

In [44]:
gw_team_joined = pd.merge(gw_player_joined, pg_team_stats, how='left', left_on='team', right_on='team', suffixes=(None, '_team'))
gw_team_joined

,GW,fixture,name,position,team,opponent,starts,minutes,goals,xG,assists,xA,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points,total_appearances,minutes_pg,goals_pg,xG_pg,assists_pg,xA_pg,GC_pg,xGC_pg,clean_sheets_pg,own_goals_pg,yellow_cards_pg,red_cards_pg,penalties_missed_pg,saves_pg,penalties_saved_pg,points_pg,goals_pg_team,xG_pg_team,GC_pg_team,xGC_pg_team,CS_pg,games_won_pg
0,1,6,Alex Scott,MID,Bournemouth,Nott'm Forest,1,62,0,0.00,0,0.01,1,1.02,0,0,0,0,0,0,0,0,2,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48
1,2,12,Alex Scott,MID,Bournemouth,Newcastle,0,26,0,0.02,0,0.01,1,1.36,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48
2,3,24,Alex Scott,MID,Bournemouth,Everton,0,24,0,0.00,0,0.01,0,0.26,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48
3,4,32,Alex Scott,MID,Bournemouth,Chelsea,0,22,0,0.00,0,0.00,1,0.32,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48
4,5,46,Alex Scott,MID,Bournemouth,Liverpool,0,21,0,0.10,0,0.02,0,0.40,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,19,187,Jakub Stolarczyk,GK,Leicester,Man City,1,90,0,0.00,0,0.00,2,1.27,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17
6735,20,191,Jakub Stolarczyk,GK,Leicester,Aston Villa,1,90,0,0.00,0,0.00,2,1.56,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17
6736,21,205,Jakub Stolarczyk,GK,Leicester,Crystal Palace,1,90,0,0.00,0,0.00,2,1.82,0,0,0,0,0,2,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17
6737,22,216,Jakub Stolarczyk,GK,Leicester,Fulham,1,90,0,0.00,0,0.00,2,2.19,0,0,0,0,0,0,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17


In [45]:
# join the pg_team_stats dataframe on opponent to get the opponent stats for each fixture
gw_opponent_joined = pd.merge(gw_team_joined, pg_team_stats, how='left', left_on='opponent', right_on='team', suffixes=(None, '_opponent'))
gw_opponent_joined

,GW,fixture,name,position,team,opponent,starts,minutes,goals,xG,assists,xA,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points,total_appearances,minutes_pg,goals_pg,xG_pg,assists_pg,xA_pg,GC_pg,xGC_pg,clean_sheets_pg,own_goals_pg,yellow_cards_pg,red_cards_pg,penalties_missed_pg,saves_pg,penalties_saved_pg,points_pg,goals_pg_team,xG_pg_team,GC_pg_team,xGC_pg_team,CS_pg,games_won_pg,team_opponent,goals_pg_opponent,xG_pg_opponent,GC_pg_opponent,xGC_pg_opponent,CS_pg_opponent,games_won_pg_opponent
0,1,6,Alex Scott,MID,Bournemouth,Nott'm Forest,1,62,0,0.00,0,0.01,1,1.02,0,0,0,0,0,0,0,0,2,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,Nott'm Forest,1.43,1.28,1.17,1.15,0.39,0.57
1,2,12,Alex Scott,MID,Bournemouth,Newcastle,0,26,0,0.02,0,0.01,1,1.36,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,Newcastle,1.74,1.76,1.13,1.24,0.39,0.52
2,3,24,Alex Scott,MID,Bournemouth,Everton,0,24,0,0.00,0,0.01,0,0.26,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,Everton,0.73,1.02,1.27,1.34,0.36,0.18
3,4,32,Alex Scott,MID,Bournemouth,Chelsea,0,22,0,0.00,0,0.00,1,0.32,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,Chelsea,1.96,2.05,1.30,1.43,0.17,0.48
4,5,46,Alex Scott,MID,Bournemouth,Liverpool,0,21,0,0.10,0,0.02,0,0.40,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,Liverpool,2.45,2.36,0.95,0.88,0.41,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,19,187,Jakub Stolarczyk,GK,Leicester,Man City,1,90,0,0.00,0,0.00,2,1.27,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,Man City,2.00,1.93,1.26,1.46,0.26,0.52
6735,20,191,Jakub Stolarczyk,GK,Leicester,Aston Villa,1,90,0,0.00,0,0.00,2,1.56,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,Aston Villa,1.43,1.50,1.52,1.25,0.13,0.43
6736,21,205,Jakub Stolarczyk,GK,Leicester,Crystal Palace,1,90,0,0.00,0,0.00,2,1.82,0,0,0,0,0,2,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,Crystal Palace,1.09,1.39,1.22,1.42,0.35,0.30
6737,22,216,Jakub Stolarczyk,GK,Leicester,Fulham,1,90,0,0.00,0,0.00,2,2.19,0,0,0,0,0,0,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,Fulham,1.43,1.42,1.30,1.17,0.17,0.35


In [46]:
# remove categorical and indetifier variables from the final model dataframe
final_model_df = gw_opponent_joined.drop(columns=["GW", "fixture", "team", "opponent", "team_opponent"])

# set the index of the df as the player, so we can map the predicted points to a player for a fixture
final_model_df = final_model_df.set_index('name')

# get dummy variables for the position column
final_model_df = pd.get_dummies(final_model_df, columns=['position'], dtype=float)

# this should be the final dataframe to use for the model
final_model_df

,starts,minutes,goals,xG,assists,xA,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,bonus,points,total_appearances,minutes_pg,goals_pg,xG_pg,assists_pg,xA_pg,GC_pg,xGC_pg,clean_sheets_pg,own_goals_pg,yellow_cards_pg,red_cards_pg,penalties_missed_pg,saves_pg,penalties_saved_pg,points_pg,goals_pg_team,xG_pg_team,GC_pg_team,xGC_pg_team,CS_pg,games_won_pg,goals_pg_opponent,xG_pg_opponent,GC_pg_opponent,xGC_pg_opponent,CS_pg_opponent,games_won_pg_opponent,position_DEF,position_FWD,position_GK,position_MID
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alex Scott,1,62,0,0.00,0,0.01,1,1.02,0,0,0,0,0,0,0,0,2,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,1.43,1.28,1.17,1.15,0.39,0.57,0.0,0.0,0.0,1.0
Alex Scott,0,26,0,0.02,0,0.01,1,1.36,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,1.74,1.76,1.13,1.24,0.39,0.52,0.0,0.0,0.0,1.0
Alex Scott,0,24,0,0.00,0,0.01,0,0.26,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,0.73,1.02,1.27,1.34,0.36,0.18,0.0,0.0,0.0,1.0
Alex Scott,0,22,0,0.00,0,0.00,1,0.32,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,1.96,2.05,1.30,1.43,0.17,0.48,0.0,0.0,0.0,1.0
Alex Scott,0,21,0,0.10,0,0.02,0,0.40,0,0,0,0,0,0,0,0,1,8,32.25,0.0,0.02,0.0,0.02,0.38,0.46,0.12,0.0,0.00,0.0,0.0,0.00,0.0,1.38,1.78,1.93,1.13,1.26,0.26,0.48,2.45,2.36,0.95,0.88,0.41,0.73,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jakub Stolarczyk,1,90,0,0.00,0,0.00,2,1.27,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,2.00,1.93,1.26,1.46,0.26,0.52,0.0,0.0,1.0,0.0
Jakub Stolarczyk,1,90,0,0.00,0,0.00,2,1.56,0,0,0,0,0,3,0,0,2,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,1.43,1.50,1.52,1.25,0.13,0.43,0.0,0.0,1.0,0.0
Jakub Stolarczyk,1,90,0,0.00,0,0.00,2,1.82,0,0,0,0,0,2,0,0,1,6,90.00,0.0,0.00,0.0,0.00,2.00,1.63,0.00,0.0,0.17,0.0,0.0,2.83,0.0,1.67,1.09,1.01,2.04,1.97,0.04,0.17,1.09,1.39,1.22,1.42,0.35,0.30,0.0,0.0,1.0,0.0


### Modelling

Train a model with the final df above and see the performance, then train a model with only the aggregated stats and exclude the fixture-specific stats and see the performance of this one in comparison.

Can use feature importance to understand which features are most influential in the model's predictions and decide whether to include fixture-specific stats in the final model based off of this.

In [47]:
# import packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [48]:
X = final_model_df.drop(columns=['points'])
y = final_model_df['points']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [50]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

LinearRegression()

In [51]:
linear_regression.coef_

array([ 6.97757677e-02,  1.39418377e-02,  4.52998394e+00, -1.60807852e-01,
        2.82657736e+00, -1.57192154e-01, -2.26367470e-01, -3.26509201e-04,
        1.99300104e+00, -2.24399353e+00, -1.03260173e+00, -3.08094793e+00,
       -1.56830010e+00,  3.04935014e-01,  5.20886014e+00,  1.23374385e+00,
        3.19041683e-03, -1.33570285e-02, -4.62626466e+00,  1.85489910e-01,
       -2.44372874e+00, -3.62904367e-01,  1.20426638e-01,  8.70955630e-02,
       -1.70371498e+00,  1.52819672e+00,  8.83450678e-01,  2.43202620e+00,
        2.59496626e+00, -2.72739819e-01, -3.89636700e+00,  7.48296833e-01,
        5.76362326e-02,  8.33443024e-02,  7.52971107e-02, -4.44416368e-02,
        1.91268157e-01, -2.63244715e-01,  6.84946786e-02, -2.41080452e-02,
       -2.67511971e-03, -7.99707939e-03,  1.05602316e-01, -2.05397028e-01,
        4.55683856e-02, -1.23432236e-01,  1.76308265e-02,  6.02330234e-02])

In [52]:
linear_regression.intercept_

0.037614998546848355

In [53]:
linear_regression.feature_names_in_

array(['starts', 'minutes', 'goals', 'xG', 'assists', 'xA', 'GC', 'xGC',
       'clean_sheets', 'own_goals', 'yellow_cards', 'red_cards',
       'penalties_missed', 'saves', 'penalties_saved', 'bonus',
       'total_appearances', 'minutes_pg', 'goals_pg', 'xG_pg',
       'assists_pg', 'xA_pg', 'GC_pg', 'xGC_pg', 'clean_sheets_pg',
       'own_goals_pg', 'yellow_cards_pg', 'red_cards_pg',
       'penalties_missed_pg', 'saves_pg', 'penalties_saved_pg',
       'points_pg', 'goals_pg_team', 'xG_pg_team', 'GC_pg_team',
       'xGC_pg_team', 'CS_pg', 'games_won_pg', 'goals_pg_opponent',
       'xG_pg_opponent', 'GC_pg_opponent', 'xGC_pg_opponent',
       'CS_pg_opponent', 'games_won_pg_opponent', 'position_DEF',
       'position_FWD', 'position_GK', 'position_MID'], dtype=object)

In [54]:
linear_regression.score(X_test, y_test)

0.9368377751879574

In [55]:
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train)
sm_model = sm.OLS(y_train, X_train_sm).fit()
sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 points   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.936
Method:                 Least Squares   F-statistic:                     1469.
Date:                Mon, 03 Mar 2025   Prob (F-statistic):               0.00
Time:                        12:08:30   Log-Likelihood:                -5272.1
No. Observations:                4717   AIC:                         1.064e+04
Df Residuals:                    4669   BIC:                         1.095e+04
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.0301      0.220      0.137      0.891      -0.401       0.461
starts                    0.0698      0.060      1.172      0.241      -0.047       0.186
minutes                   0.0139      0.001     14.201      0.000       0.012       0.016
goals                     4.5300      0.054     83.900      0.000       4.424       4.636
xG                       -0.1608      0.076     -2.114      0.035      -0.310      -0.012
assists                   2.8266      0.041     69.315      0.000       2.747       2.907
xA                       -0.1572      0.111     -1.420      0.156      -0.374       0.060
GC                       -0.2264      0.016    -14.223      0.000      -0.258      -0.195
xGC                      -0.0003      0.022     -0.015      0.988      -0.043       0.042
clean_sheets              1.9930      0.039     50.548      0.000       1.916       2.070
own_goals                -2.2440      0.215    -10.439      0.000      -2.665      -1.823
yellow_cards             -1.0326      0.033    -31.624      0.000      -1.097      -0.969
red_cards                -3.0809      0.157    -19.563      0.000      -3.390      -2.772
penalties_missed         -1.5683      0.344     -4.561      0.000      -2.242      -0.894
saves                     0.3049      0.022     14.026      0.000       0.262       0.348
penalties_saved           5.2089      0.349     14.940      0.000       4.525       5.892
bonus                     1.2337      0.024     52.275      0.000       1.187       1.280
total_appearances         0.0032      0.003      1.073      0.283      -0.003       0.009
minutes_pg               -0.0134      0.002     -6.308      0.000      -0.018      -0.009
goals_pg                 -4.6263      0.313    -14.773      0.000      -5.240      -4.012
xG_pg                     0.1855      0.259      0.717      0.474      -0.322       0.693
assists_pg               -2.4437      0.213    -11.487      0.000      -2.861      -2.027
xA_pg                    -0.3629      0.329     -1.102      0.271      -1.009       0.283
GC_pg                     0.1204      0.091      1.323      0.186      -0.058       0.299
xGC_pg                    0.0871      0.112      0.780      0.435      -0.132       0.306
clean_sheets_pg          -1.7037      0.223     -7.647      0.000      -2.141      -1.267
own_goals_pg              1.5282      0.818      1.869      0.062      -0.075       3.131
yellow_cards_pg           0.8835      0.125      7.051      0.000       0.638       1.129
red_cards_pg              2.4320      0.662      3.672      0.000       1.134       3.730
penalties_missed_pg       2.5950      1.598      1.624      0.104      -0.537       5.727
saves_pg                 -0.2727      0.068     -4.022      0.000      -0.406      -0.140
penalties_saved_pg     

In [56]:
print(X_train_sm.dtypes)

const                    float64
starts                     int64
minutes                    int64
goals                      int64
xG                       float64
assists                    int64
xA                       float64
GC                         int64
xGC                      float64
clean_sheets               int64
own_goals                  int64
yellow_cards               int64
red_cards                  int64
penalties_missed           int64
saves                      int64
penalties_saved            int64
bonus                      int64
total_appearances          int32
minutes_pg               float64
goals_pg                 float64
xG_pg                    float64
assists_pg               float64
xA_pg                    float64
GC_pg                    float64
xGC_pg                   float64
clean_sheets_pg          float64
own_goals_pg             float64
yellow_cards_pg          float64
red_cards_pg             float64
penalties_missed_pg      float64
saves_pg  

In [57]:
print(X_train_sm.isnull().sum())

const                    0
starts                   0
minutes                  0
goals                    0
xG                       0
assists                  0
xA                       0
GC                       0
xGC                      0
clean_sheets             0
own_goals                0
yellow_cards             0
red_cards                0
penalties_missed         0
saves                    0
penalties_saved          0
bonus                    0
total_appearances        0
minutes_pg               0
goals_pg                 0
xG_pg                    0
assists_pg               0
xA_pg                    0
GC_pg                    0
xGC_pg                   0
clean_sheets_pg          0
own_goals_pg             0
yellow_cards_pg          0
red_cards_pg             0
penalties_missed_pg      0
saves_pg                 0
penalties_saved_pg       0
points_pg                0
goals_pg_team            0
xG_pg_team               0
GC_pg_team               0
xGC_pg_team              0
C

### Stat Leaders and Functions

In [58]:
# remove players who have not played 60 minutes per game as they are not regular starters and I am not interested in them
# After 60 minutes a player gets extra points, so I only want players who play after this
#player_total_stats = player_stats.loc[(player_stats['minutes_per_game'] >= 60)]
#player_total_stats

In [59]:
goalkeepers = player_stats.loc[player_stats['position'] == 'GK']
defenders = player_stats.loc[player_stats['position'] == 'DEF']
midfielders = player_stats.loc[player_stats['position'] == 'MID']
forwards = player_stats.loc[player_stats['position'] == 'FWD']

In [60]:
#team_stats_per_gw.head(10)

Functions for Decision Making

* Make function to find what team has best fixtures over x amount of games
* What players/teams have gotten the most x stat over the last y amount of games

In [61]:
# def team_stats_function(starting_gameweek: int, ending_gameweek: int, stats: list):
        
#         filtered_gameweeks = team_stats_per_gw.loc[(team_stats_per_gw['GW'] >= starting_gameweek) & (team_stats_per_gw['GW'] <= ending_gameweek)]
#         team_result = filtered_gameweeks.groupby('team').sum()[['goals_scored', 'xG', 'goals_home', 'xG_home', 'goals_away','xG_away', 'goals_conceded', 'xGC',
#                                                                 'goals_conceded_home', 'xGC_home', 'goals_conceded_away','xGC_away', 'clean_sheet']].reset_index()
#         sort_team_result = team_result.sort_values(stats, ascending=False).reset_index(drop=True)
        
#         return sort_team_result

In [62]:
#team_stats_function(15, 22, 'xG')